In [ ]:
# default_exp klauswert


# klauswert

> Ein Modul zur Klausurauswertung

In [ ]:
#hide
from nbdev.showdoc import *
from IPython.display import Markdown,display

In [ ]:
#hide
# Modificated version of method show_doc more suitable for LaTeX documents
def my_show_doc(item):
    display(Markdown(show_doc(item, disp=False).replace('[source]','')))
    
def my_dispMarkdown(item):
    display(Markdown(item.to_markdown()))

## Module einbinden

In [ ]:
# exports
import pandas as pd
import yaml


## Klasse "Klauswert"

### Standard Konfiguration

In [ ]:
# exports
stdConfig = '''headerPC: 4
pointCols: "A:L"
headerMC: 1
markCols: "N:O"
headerMP: 1
mxPointCols:  "A:L"
'''

In [ ]:
#export
class Klauswert():
    '''Aufgabe der Klasse: Klausurauswertung \\
      Konfiguration: YAML-Text (stdConfig)\\
      Ausgangslage: .xlsx-Datei mit einer Zeile je Student. In den Spalten  
      config['pointCols'] stehen die Punkte für die jeweiligen Aufgaben. In den Spalten
      config['markCols'] ist die Notenverteilung ersichtlich
    '''
    def __init__(self, fname, configTxt = stdConfig):
        self.config = yaml.load(configTxt, Loader=yaml.FullLoader)
        
        # Aufgabenpunkte der Studierenden laden
        self.dfKerg=pd.read_excel(fname, header=self.config['headerPC'], 
                                  usecols=self.config['pointCols'])
        #display(self.dfKerg.head())
        
        # max. mögliche Aufgabenpunkte laden
        self.dfPkt=pd.read_excel(fname, header=self.config['headerMP'], 
                                  usecols=self.config['mxPointCols'], nrows=1)
        
        # Prozentschwellen für Noten laden
        self.ndf=pd.read_excel(fname, header=self.config['headerMC'], 
                                  usecols=self.config['markCols']).dropna()
        #display(self.ndf.head())
        # Gesamtpunkte pro Student
        self.dfKerg['sumP'] = self.dfKerg.sum(axis=1)

        # Zielerreichung pro Student
        sumP=self.dfPkt.sum(axis=1)[0]
        self.dfKerg2=self.dfKerg.copy()
        self.dfKerg2['pktProz'] = self.dfKerg2['sumP']/sumP*100
        # Noten berechnen
        self.dfKerg2['Note'] =  self.dfKerg2['pktProz'].map(lambda proz: self.__noteB(proz))

    
    def __noteB(self, punkte):
        return self.ndf[self.ndf['%'] <= punkte].iloc[0]['Note']

    def getConfig(self):
        return self.config
    
    def getAllData(self):
        '''Aufgabe: gesamten Dataframe zurück geben.'''
        return self.dfKerg
    
    def getNoten(self):
        return self.dfKerg2[['Name', 'Note']]
    
    def getNotenDefinition(self):
        return self.ndf
    
    def getDurchgefallen(self):
        '''Liefert eine Liste mit den Namen der Teilnehmer, die die Klausur nicht bestanden haben, 
        sowie deren Zielerreichungswert.
        '''
        return self.dfKerg2[self.dfKerg2['Note'] > 4][['Name', 'pktProz', 'Note']]

In [ ]:
pbx2020 = Klauswert('rawData/KlausurPunkte.xlsx')


In [ ]:
my_show_doc(Klauswert)

<h2 id="Klauswert" class="doc_header"><code>class</code> <code>Klauswert</code><a href="" class="source_link" style="float:right"></a></h2>

> <code>Klauswert</code>(**`fname`**, **`configTxt`**=*`'headerPC: 4\npointCols: "A:L"\nheaderMC: 1\nmarkCols: "N:O"\nheaderMP: 1\nmxPointCols:  "A:L"\n'`*)

Aufgabe der Klasse: Klausurauswertung \
Konfiguration: YAML-Text (stdConfig)\
Ausgangslage: .xlsx-Datei mit einer Zeile je Student. In den Spalten  
config['pointCols'] stehen die Punkte für die jeweiligen Aufgaben. In den Spalten
config['markCols'] ist die Notenverteilung ersichtlich

In [ ]:
pbx2020 = Klauswert('rawData/KlausurPunkte.xlsx')
print(pbx2020.getConfig()['headerMC'])

1


### Die Funktionen

In [ ]:
my_show_doc(Klauswert.getAllData)

<h4 id="Klauswert.getAllData" class="doc_header"><code>Klauswert.getAllData</code><a href="__main__.py#L42" class="source_link" style="float:right"></a></h4>

> <code>Klauswert.getAllData</code>()

Aufgabe: gesamten Dataframe zurück geben.

In [ ]:
display(Markdown(pbx2020.getAllData().head().to_markdown()))

|    | Name        |   A1 |   A2 |   A3 |   A4 |   A5 |   A6 |   A7 |   A8 |   A9 |   A10 |   A11 |   sumP |
|---:|:------------|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|------:|------:|-------:|
|  0 | Alexander   |  4   |  6   |    4 |  7   | 11   |  4   |    4 |  5   |  5   |   4.5 |  10.5 |   65   |
|  1 | Andre       |  4   |  3.5 |    4 |  8   | 20.5 |  4   |    3 |  3.5 |  3.5 |   3   |   4   |   61   |
|  2 | Celine      |  4   |  3.5 |    2 |  7   | 13.5 |  5   |    5 |  5   |  4.5 |   5   |  11   |   65.5 |
|  3 | Christian   |  3.5 |  2.5 |    1 |  6.5 | 17   |  2.5 |    5 |  5   |  4   |   6   |   4   |   57   |
|  4 | Christopher |  3.5 |  0.5 |    0 |  4   |  7.5 |  0   |    0 |  0   |  0   |   0   |   0   |   15.5 |

Die folgende Tabelle ist eine Kopie der Codeausgabe der vorherigen Zelle:

|    | Name        |   A1 |   A2 |   A3 |   A4 |   A5 |   A6 |   A7 |   A8 |   A9 |   A10 |   A11 |   sumP |
|---:|:------------|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|------:|------:|-------:|
|  0 | Alexander   |  4   |  6   |    4 |  7   | 11   |  4   |    4 |  5   |  5   |   4.5 |  10.5 |   65   |
|  1 | Andre       |  4   |  3.5 |    4 |  8   | 20.5 |  4   |    3 |  3.5 |  3.5 |   3   |   4   |   61   |
|  2 | Celine      |  4   |  3.5 |    2 |  7   | 13.5 |  5   |    5 |  5   |  4.5 |   5   |  11   |   65.5 |
|  3 | Christian   |  3.5 |  2.5 |    1 |  6.5 | 17   |  2.5 |    5 |  5   |  4   |   6   |   4   |   57   |
|  4 | Christopher |  3.5 |  0.5 |    0 |  4   |  7.5 |  0   |    0 |  0   |  0   |   0   |   0   |   15.5 |

In [ ]:
dfKerg=pd.read_excel('rawData/KlausurPunkte.xlsx', header=4, usecols='A:L')

In [ ]:
my_dispMarkdown(dfKerg.head())

|    | Name        |   A1 |   A2 |   A3 |   A4 |   A5 |   A6 |   A7 |   A8 |   A9 |   A10 |   A11 |
|---:|:------------|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|------:|------:|
|  0 | Alexander   |  4   |  6   |    4 |  7   | 11   |  4   |    4 |  5   |  5   |   4.5 |  10.5 |
|  1 | Andre       |  4   |  3.5 |    4 |  8   | 20.5 |  4   |    3 |  3.5 |  3.5 |   3   |   4   |
|  2 | Celine      |  4   |  3.5 |    2 |  7   | 13.5 |  5   |    5 |  5   |  4.5 |   5   |  11   |
|  3 | Christian   |  3.5 |  2.5 |    1 |  6.5 | 17   |  2.5 |    5 |  5   |  4   |   6   |   4   |
|  4 | Christopher |  3.5 |  0.5 |    0 |  4   |  7.5 |  0   |    0 |  0   |  0   |   0   |   0   |

In [ ]:
my_show_doc(Klauswert.getDurchgefallen)

<h4 id="Klauswert.getDurchgefallen" class="doc_header"><code>Klauswert.getDurchgefallen</code><a href="__main__.py#L52" class="source_link" style="float:right"></a></h4>

> <code>Klauswert.getDurchgefallen</code>()

Liefert eine Liste mit den Namen der Teilnehmer, die die Klausur nicht bestanden haben, 
sowie deren Zielerreichungswert.

In [ ]:
my_dispMarkdown(pbx2020.getDurchgefallen().head())

|    | Name        |   pktProz |   Note |
|---:|:------------|----------:|-------:|
|  4 | Christopher |   17.2222 |      5 |
| 15 | Kamil       |   25      |      5 |
| 20 | Lukas       |   45      |      5 |
| 34 | Tim         |   47.2222 |      5 |
| 35 | Tobias      |   44.4444 |      5 |

|    | Name        |   pktProz |   Note |
|---:|:------------|----------:|-------:|
|  4 | Christopher |   17.2222 |      5 |
| 15 | Kamil       |   25      |      5 |
| 20 | Lukas       |   45      |      5 |
| 34 | Tim         |   47.2222 |      5 |
| 35 | Tobias      |   44.4444 |      5 |

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 10_Code.ipynb.
Converted 11_FirstExample.ipynb.
Converted 20_Examples.ipynb.
Converted 30_CodeExample.ipynb.
Converted 31_klauswertTests.ipynb.
Converted index.ipynb.
